In [3]:
import pandas as pd
import numpy as np


In [324]:

kg = pd.read_csv("CSVs/Kaggle Data/NBA_PBP_2020-21.csv")
kg["play_count"] = 1
kg["season"]= "2020-2021"
kg['ShotPoints'] = kg['ShotType'].str[:4]
#kg['GameStartDTS'] = kg.assign(GameStartDateTime=lambda kg : pd.to_datetime(kg.Date + ' ' + kg.Time))
kg['GameID'] = kg['Date'] + kg['HomeTeam'] + kg['AwayTeam']

condlist = (kg['FreeThrowOutcome']=='make',kg['ShotPoints']=='2-pt',kg['ShotPoints']=='3-pt')
vallist = (1,2,3)
kg['Points'] = np.select(condlist,vallist,default=0)
kg['ShotVersion'] = np.select(
[
    (kg['ShotPoints']=='2-pt'),(kg['ShotPoints']=='3-pt'),(pd.notnull(kg['FreeThrowOutcome']))
],
[    ('fg'),('3'),('ft')
],default='NaN'
)

kg['ShotSuccess'] = np.select(
[
    ((kg['FreeThrowOutcome']=='make')|(kg['ShotOutcome']=='make')),((kg['FreeThrowOutcome']=='miss')|(kg['ShotOutcome']=='miss'))
],
[
    (1),(0)
],default=0
)

kg['PlayType'] = np.select(
[    
    (pd.notnull(kg['FreeThrowOutcome']))
     ,(pd.notnull(kg['ShotOutcome']))
     ,(pd.notnull(kg['Rebounder']))
     ,(pd.notnull(kg['FoulType']))
     ,(pd.notnull(kg['JumpballAwayPlayer']))
     ,(pd.notnull(kg['TurnoverType']))
     ,(pd.notnull(kg['EnterGame']))
     ,(pd.notnull(kg['TimeoutTeam']))
     ,(pd.notnull(kg['ViolationType']))
     ,((kg['AwayPlay'].str[:3]=='End')|(kg['HomePlay'].str[:3]=='End'))
     ,((kg['AwayPlay'].str[:7]=='Instant')|(kg['HomePlay'].str[:7]=='Instant'))
     ,((kg['AwayPlay'].str[:5]=='Def 3')|(kg['HomePlay'].str[:5]=='Def 3'))
],
[
    ('FT')
    ,('Shot')
    ,('Rebound')
    ,('Foul')
    ,('Jump Ball')
    ,('Turnover')
    ,('Sub')
    ,('Timeout')
    ,('Violation')
    ,('End of Q')
    ,('Replay')
    ,('Def 3 Sec') 
],default='Other'
)

kg['Live Play'] = np.select(
[
    ((kg['PlayType']=='Shot')|(kg['PlayType']=='Turnover')|(kg['PlayType']=='Foul')|(kg['PlayType']=='Rebound'))
],
[
    (1)
],default=0
)

kg['Trip to the Line'] = np.select(
[
    (pd.notnull(kg['FreeThrowNum']))&(kg['FreeThrowNum']!='1 of 1')
],
[
    1
],default=0
)

kg['Player'] = np.select(
[
    kg['PlayType']=='Shot',kg['PlayType']=='Turnover',kg['Trip to the Line']==1,kg['PlayType']=='Rebound',kg['PlayType']=='Foul'
],
[
    kg['Shooter'],kg['TurnoverPlayer'],kg['FreeThrowShooter'],kg['Rebounder'],kg['Fouler']
]
)

kg['Shot Distance Type'] = np.select(
[
    kg['ShotDist']<5,(kg['ShotDist']>=5)|(kg['ShotDist']<=10),(kg['ShotDist']>10)|(kg['ShotPoints']=='2-pt'),kg['ShotPoints']=='3-pt'
],
[
    'Within 5 Feet','Short Mid Range','Long Mid Range','3 Point Range'
],default='NaN'
)

kg['Points'] = kg['Points'].astype('int64')
kg['ShotSuccess'] = kg['ShotSuccess'].astype('int64')



kg


,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,ShotPoints,GameID,Points,ShotVersion,ShotSuccess,PlayType,Live Play,Trip to the Line,Player,Shot Distance Type
0,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,720,GSW,Jump ball: J. Wiseman vs. D. Jordan (J. Harris...,...,NaN,December 22 2020BRKGSW,0,NaN,0,Jump Ball,0,0,0,NaN
1,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,710,GSW,NaN,...,NaN,December 22 2020BRKGSW,0,NaN,0,Turnover,1,0,D. Jordan - jordade01,NaN
2,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,Shooting foul by K. Irving (drawn by S. Curry),...,NaN,December 22 2020BRKGSW,0,NaN,0,Foul,1,0,K. Irving - irvinky01,NaN
3,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,S. Curry makes free throw 1 of 2,...,NaN,December 22 2020BRKGSW,1,ft,1,FT,0,1,S. Curry - curryst01,NaN
4,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,S. Curry makes free throw 2 of 2,...,NaN,December 22 2020BRKGSW,1,ft,1,FT,0,1,S. Curry - curryst01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97668,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,15,ATL,NaN,...,2-pt,January 15 2021UTAATL,2,fg,0,Shot,1,0,S. Harrison - harrish01,Within 5 Feet
97669,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,13,ATL,NaN,...,NaN,January 15 2021UTAATL,0,NaN,0,Rebound,1,0,U. Azubuike - azubuud01,NaN
97670,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,10,ATL,NaN,...,NaN,January 15 2021UTAATL,0,NaN,0,Turnover,1,0,U. Azubuike - azubuud01,NaN
97671,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,0,ATL,End of 4th quarter,...,NaN,January 15 2021UTAATL,0,NaN,0,End of Q,0,0,0,NaN


In [325]:
# Code below is to help infer starters per team per game
## If the first Sub action for the player per game is LeaveGame and not EnterGame then they were a starter
### Looking to add 5 columns to represent players for each team so each play has 10 associated players
### Will also add a EnterGame action for each of the 5 starters in each game for easy minutes calculations

####This doesn't work :/ Unless the code is flawed it seems like the substitution actions have an issue in the data set. The number of players who's first sub actino is to leave the game is inconsistent

subs = kg.loc[pd.notnull(kg['EnterGame'])
              ,['GameID','Quarter','Date','Time','HomeTeam','AwayTeam','SecLeft','EnterGame','LeaveGame','AwayPlay','HomePlay']].sort_values(['Date','Time','HomeTeam','AwayTeam','Quarter','SecLeft']
    ,ascending=[True,True,True,True,True,False])


##the code below isn't the best way to do this, can come back and clean it up
he = subs.loc[pd.notnull(subs['HomePlay']),['GameID','Quarter','HomeTeam','SecLeft','EnterGame']]
he['PlayerAction'] ='Enter'
he['Team'] = he['HomeTeam']
he['Player'] = he['EnterGame']

ae = subs.loc[pd.notnull(subs['AwayPlay']),['GameID','Quarter','AwayTeam','SecLeft','EnterGame']]
ae['PlayerAction'] ='Enter'
ae['Team'] = ae['AwayTeam']
ae['Player'] = ae['EnterGame']

al = subs.loc[pd.notnull(subs['AwayPlay']),['GameID','Quarter','AwayTeam','SecLeft','LeaveGame']]
al['PlayerAction'] ='Leave'
al['Team'] = al['AwayTeam']
al['Player'] = al['LeaveGame']

hl = subs.loc[pd.notnull(subs['HomePlay']),['GameID','Quarter','HomeTeam','SecLeft','LeaveGame']]
hl['PlayerAction'] ='Leave'
hl['Team'] = hl['HomeTeam']
hl['Player'] = hl['LeaveGame']

#would be nice not to have the unneccessary columns here but not a big deal
sub = pd.concat([he,ae,al,hl])


sub = sub.sort_values(['GameID','Quarter','SecLeft'],ascending=[True,True,False])   #loc[:,['GameID','SecLeft','Player','PlayerAction']].
#so = sub.groupby(['GameID','Player']).cumcount()+1
sub['SubOrder'] = sub.groupby(['GameID','Player']).cumcount()+1
#subo = pd.concat([sub,so],axis=1)
sub

#sub.to_csv("test_order.csv")


,GameID,Quarter,HomeTeam,SecLeft,EnterGame,PlayerAction,Team,Player,AwayTeam,LeaveGame,SubOrder
46,December 22 2020BRKGSW,1,NaN,440,K. Looney - looneke01,Enter,GSW,K. Looney - looneke01,GSW,NaN,1
46,December 22 2020BRKGSW,1,NaN,440,NaN,Leave,GSW,J. Wiseman - wisemja01,GSW,J. Wiseman - wisemja01,1
61,December 22 2020BRKGSW,1,BRK,360,J. Allen - allenja01,Enter,BRK,J. Allen - allenja01,NaN,NaN,1
60,December 22 2020BRKGSW,1,NaN,360,K. Bazemore - bazemke01,Enter,GSW,K. Bazemore - bazemke01,GSW,NaN,1
60,December 22 2020BRKGSW,1,NaN,360,NaN,Leave,GSW,A. Wiggins - wiggian01,GSW,A. Wiggins - wiggian01,1
...,...,...,...,...,...,...,...,...,...,...,...
82806,January 9 2021WASMIA,4,NaN,462,NaN,Leave,MIA,G. Dragić - dragigo01,MIA,G. Dragić - dragigo01,5
82815,January 9 2021WASMIA,4,NaN,405,K. Olynyk - olynyke01,Enter,MIA,K. Olynyk - olynyke01,MIA,NaN,4
82815,January 9 2021WASMIA,4,NaN,405,NaN,Leave,MIA,A. Iguodala - iguodan01,MIA,A. Iguodala - iguodan01,4
82836,January 9 2021WASMIA,4,WAS,314,D. Avdija - avdijde01,Enter,WAS,D. Avdija - avdijde01,NaN,NaN,4


In [326]:
pg = kg #.loc[]
pg['FullPlay'] = pg[['AwayPlay','HomePlay']].bfill(axis=1).iloc[:,0]

pg['Team'] = np.select(
[pd.notnull(pg['AwayPlay']),pd.notnull(['HomePlay'])],
[pg['AwayTeam'],pg['HomeTeam']])

pg['OpposingTeam'] = np.select(
[pd.notnull(pg['AwayPlay']),pd.notnull(['HomePlay'])],
[pg['HomeTeam'],pg['AwayTeam']])

pg

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,ShotVersion,ShotSuccess,PlayType,Live Play,Trip to the Line,Player,Shot Distance Type,FullPlay,Team,OpposingTeam
0,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,720,GSW,Jump ball: J. Wiseman vs. D. Jordan (J. Harris...,...,NaN,0,Jump Ball,0,0,0,NaN,Jump ball: J. Wiseman vs. D. Jordan (J. Harris...,GSW,BRK
1,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,710,GSW,NaN,...,NaN,0,Turnover,1,0,D. Jordan - jordade01,NaN,Turnover by D. Jordan (bad pass),BRK,GSW
2,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,Shooting foul by K. Irving (drawn by S. Curry),...,NaN,0,Foul,1,0,K. Irving - irvinky01,NaN,Shooting foul by K. Irving (drawn by S. Curry),GSW,BRK
3,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,S. Curry makes free throw 1 of 2,...,ft,1,FT,0,1,S. Curry - curryst01,NaN,S. Curry makes free throw 1 of 2,GSW,BRK
4,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,S. Curry makes free throw 2 of 2,...,ft,1,FT,0,1,S. Curry - curryst01,NaN,S. Curry makes free throw 2 of 2,GSW,BRK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97668,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,15,ATL,NaN,...,fg,0,Shot,1,0,S. Harrison - harrish01,Within 5 Feet,S. Harrison misses 2-pt layup from 3 ft,UTA,ATL
97669,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,13,ATL,NaN,...,NaN,0,Rebound,1,0,U. Azubuike - azubuud01,NaN,Offensive rebound by U. Azubuike,UTA,ATL
97670,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,10,ATL,NaN,...,NaN,0,Turnover,1,0,U. Azubuike - azubuud01,NaN,Turnover by U. Azubuike (bad pass; steal by O....,UTA,ATL
97671,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,0,ATL,End of 4th quarter,...,NaN,0,End of Q,0,0,0,NaN,End of 4th quarter,ATL,UTA


In [327]:
a = pg.loc[pd.notnull(pg['Assister'])]
a['PlayType'] = 'Assist'
a['Player'] = a['Assister']
a
pga = pd.concat([pg,a])
#pga.loc[pd.isnull(pga['Player'])]
pga

/var/folders/gt/htm9567s4_g0s4y6py45x30w0000gp/T/ipykernel_51146/1414438297.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['PlayType'] = 'Assist'
/var/folders/gt/htm9567s4_g0s4y6py45x30w0000gp/T/ipykernel_51146/1414438297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['Player'] = a['Assister']


,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,ShotVersion,ShotSuccess,PlayType,Live Play,Trip to the Line,Player,Shot Distance Type,FullPlay,Team,OpposingTeam
0,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,720,GSW,Jump ball: J. Wiseman vs. D. Jordan (J. Harris...,...,NaN,0,Jump Ball,0,0,0,NaN,Jump ball: J. Wiseman vs. D. Jordan (J. Harris...,GSW,BRK
1,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,710,GSW,NaN,...,NaN,0,Turnover,1,0,D. Jordan - jordade01,NaN,Turnover by D. Jordan (bad pass),BRK,GSW
2,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,Shooting foul by K. Irving (drawn by S. Curry),...,NaN,0,Foul,1,0,K. Irving - irvinky01,NaN,Shooting foul by K. Irving (drawn by S. Curry),GSW,BRK
3,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,S. Curry makes free throw 1 of 2,...,ft,1,FT,0,1,S. Curry - curryst01,NaN,S. Curry makes free throw 1 of 2,GSW,BRK
4,/boxscores/202012220BRK.html,regular,Barclays Center Brooklyn New York,December 22 2020,7:00 PM,BRK,1,698,GSW,S. Curry makes free throw 2 of 2,...,ft,1,FT,0,1,S. Curry - curryst01,NaN,S. Curry makes free throw 2 of 2,GSW,BRK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97638,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,213,ATL,B. Fernando makes 2-pt dunk from 2 ft (assist ...,...,fg,1,Assist,1,0,B. Goodwin - goodwbr01,Within 5 Feet,B. Fernando makes 2-pt dunk from 2 ft (assist ...,ATL,UTA
97654,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,133,ATL,NaN,...,fg,1,Assist,1,0,E. Hughes - hugheel01,Short Mid Range,G. Niang makes 2-pt layup from 5 ft (assist by...,UTA,ATL
97657,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,109,ATL,NaN,...,fg,1,Assist,1,0,G. Niang - niangge01,Short Mid Range,E. Hughes makes 2-pt jump shot from 23 ft (ass...,UTA,ATL
97660,/boxscores/202101150UTA.html,regular,Vivint Smart Home Arena Salt Lake City Utah,January 15 2021,9:00 PM,UTA,4,88,ATL,NaN,...,3,1,Assist,1,0,G. Niang - niangge01,Short Mid Range,M. Oni makes 3-pt jump shot from 25 ft (assist...,UTA,ATL


In [328]:
pga.to_csv('CSVs/NBA_PBP.csv', index=False)